# Implement TORUS by using CNV data of Schizophrenia in Sweden
### Use Sweden Schizphrenia individual data to obtain gene and CNV overlap configuration; columns represent genes, rows represent individuals
### Use the transformed file to perform TOURS

#### Obtain file format used in TORUS

Use 

In [1]:
import pandas as pd
from pandasql import sqldf
from utils import load_reference_gene

In [2]:
# load reference genes and Sweden CNV data
ref_genes = load_reference_gene("../data/refGene.txt.gz")
sw_cnv = pd.read_table("../data/swcnv/swcnv.qc6.cnv", sep="\s+")
sw_indiv = pd.read_table("../data/swcnv/swcnv.qc6.cnv.indiv", sep="\s+") # "NSEG": number of CNV in each sample
sw_cnv_indiv = pd.merge(sw_cnv, sw_indiv, how="inner", on=["FID", "IID"])
sw_cnv_indiv["PHE"] = sw_cnv_indiv.apply(lambda row: 1 if row["PHE"]==2 else 0, axis=1)

In [3]:
# calcium pathway genes
calpath_genes = pd.read_table("../data/calciumgeneset.txt", skiprows = 2, header = None, names = ["gene_name"])
calpath_genes_list = calpath_genes["gene_name"].tolist()
calpath_genes_pos = pd.merge(ref_genes, calpath_genes, how="inner", on=["gene_name"])

In [4]:
query = '''
SELECT gene_name, chrom, min(tx_start), max(tx_end)
FROM calpath_genes_pos
GROUP BY gene_name
'''
calpath_genes_pos = sqldf(query)
calpath_genes_pos = calpath_genes_pos.rename(columns={"min(tx_start)": "gene_start", "max(tx_end)": "gene_end"})

In [5]:
diff = list(set(calpath_genes_list) - set(calpath_genes_pos["gene_name"].tolist()))
calpath_genes_pos.loc[calpath_genes_pos.shape[0]] = [diff[0], "chr2", 65205076, 65206278]
calpath_genes_pos.loc[calpath_genes_pos.shape[0]] = [diff[1], "chr15", 41231149, 41281887]
calpath_genes_pos["chr"] = calpath_genes_pos.apply(lambda row: int(row["chrom"][3:]) 
                                                   if not row["chrom"][3:] in ("X", "Y") else 23, axis=1)

In [6]:
print (calpath_genes_pos)

     gene_name  chrom  gene_start   gene_end  chr
0        ADCY1   chr7    45613738   45762714    7
1        ADCY2   chr5     7396342    7830194    5
2        ADCY3   chr2    25042038   25142886    2
3        ADCY4  chr14    24787554   24804277   14
4        ADCY7  chr16    50300450   50352045   16
5        ADCY8   chr8   131792546  132052835    8
6        ADCY9  chr16     4012649    4166186   16
7      ADORA2A  chr22    24819564   24838328   22
8      ADORA2B  chr17    15848230   15879210   17
9       ADRA1A   chr8    26605666   26724760    8
10      ADRA1B   chr5   159343739  159400017    5
11      ADRA1D  chr20     4201277    4229659   20
12       ADRB1  chr10   115803805  115806667   10
13       ADRB2   chr5   148206155  148208197    5
14       ADRB3   chr8    37820513   37824184    8
15       AGTR1   chr3   148415657  148460790    3
16      ATP2A1  chr16    28889808   28915830   16
17      ATP2A2  chr12   110719031  110788897   12
18      ATP2A3  chr17     3827162    3867758   17


In [7]:
# need to change
query = '''
SELECT sw.CHR, sw.BP1, sw.BP2, sw.TYPE, sw.PHE, cal.gene_name, cal.gene_start, cal.gene_end, count(sw.FID)
FROM calpath_genes_pos cal LEFT JOIN sw_cnv_indiv sw
WHERE sw.CHR == cal.chr
AND (
(sw.BP1 >= cal.gene_start AND sw.BP1 <= cal.gene_end)
OR
(sw.BP2 >= cal.gene_start AND sw.BP2 <= cal.gene_end)
OR
(sw.BP1 <= cal.gene_start AND sw.BP2 >= cal.gene_end)
OR
(sw.BP1 >= cal.gene_start AND sw.BP2 <= cal.gene_end)
)
GROUP BY sw.CHR, sw.BP1, sw.BP2, cal.gene_name, sw.TYPE, sw.PHE
'''
# overlap_CNV_gene_df = sqldf(query).drop_duplicates(subset=("CHR", "BP1", "BP2", "gene_name", "TYPE", "PHE", "count(sw.FID)"))
overlap_CNV_gene = sqldf(query)
overlap_CNV_gene = overlap_CNV_gene.rename(columns={"count(sw.FID)": "n_FID"})

In [8]:
overlap_CNV_gene

,CHR,BP1,BP2,TYPE,PHE,gene_name,gene_start,gene_end,n_FID
0,1,1804302,2047584,3,0,CALML6,1846265,1848733,1
1,1,1804302,2047584,3,1,CALML6,1846265,1848733,1
2,1,226748623,226858419,1,1,ITPKB,226819390,226926876,1
3,1,237151125,237493362,1,1,RYR2,237205701,237997288,1
4,1,239678115,239800921,3,0,CHRM3,239792372,240072717,1
5,2,40450245,40614668,1,1,SLC8A1,40339285,40739575,1
6,2,40674308,41559590,3,0,SLC8A1,40339285,40739575,1
7,2,74969763,75323267,3,0,TACR1,75273589,75426645,1
8,2,183023392,183126288,1,1,PDE1A,183004761,183387572,1
9,2,218749319,219480897,3,0,PLCD4,219472487,219501909,1
